# Exploration and Preprocessing Patrick

In [37]:
# Import relevant packages
import pandas as pd
import numpy as np
import os

## Data Loading

In [38]:
# List the folders containing the text files
folders = ['../data/raw/txt_files_0_1750', '../data/raw/txt_files_1751_3500', '../data/raw/txt_files_3501_5000']

# Initialize an empty list to store the data
data = []

# Iterate through each folder
for folder in folders:
    # Get the list of text files in the folder
    file_list = os.listdir(folder)
    
    # Iterate through each text file
    for file_name in file_list:
        # Read the contents of the file
        file_path = os.path.join(folder, file_name)
        try:
            with open(file_path, 'r') as file:
                content = file.read()
                # Append content to the data list
                data.append(content)
        except FileNotFoundError:
            print(f"File {file_name} not found.")
        except IOError:
            print(f"Error reading the file {file_name}.")
        
# Create a DataFrame from the data list
df = pd.DataFrame(data, columns=['Content'])
df


,Content
0,ARIVE | Your favorite brandsGet the appOur Vis...
1,Rückabwicklung - von Lebensversicherungen\nWir...
2,\nStarke Versicherungen für deinen Lifestyle...
3,\nRESTUBE - the airbag for more freedom and s...
4,CureVac - Wir revolutionieren die mRNA für das...
...,...
3275,\n LEMONSGATE ...
3276,Safily\n \n√úberspringen\nSuchen\nSuche schlie...
3277,Startseite | ICO-LUX\n \nFraud Prevention\nÜbe...
3278,Versus | Compare everything\nCategoriessmartph...


# Data Cleaning

## Preprocesssing Functions from the exercise

In [39]:
# import packages
import re

# import pandas as pd
# import numpy as np

# import matplotlib.pyplot as plt
# import seaborn as sns

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer


from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB

# from skmultilearn.problem_transform import BinaryRelevance

In [40]:
# define functions
def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext

def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'', sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ', cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned

def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

def removeStopWords(sentence, stopwords):
    re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)   
    return re_stop_words.sub(" ", sentence)

def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

# define stopwords
stop_words = set(stopwords.words(['german', 'english']))

# define stemmer
stemmer = SnowballStemmer('german', 'english')

# apply functions
df['Content'] = df['Content'].str.lower()
df['Content'] = df['Content'].apply(cleanHtml)
df['Content'] = df['Content'].apply(cleanPunc)
df['Content'] = df['Content'].apply(keepAlpha)
df['Content'] = df['Content'].apply(removeStopWords, stopwords=stopwords)
df['Content'] = df['Content'].apply(stemming)

In [41]:
# check result
df.head()

,Content
0,ariv favorit brandsget appour visionpartnershi...
1,r ckabwickl lebensversicher hol doppelt leb re...
2,stark versicher f r lifestyl held de hom produ...
3,restub airbag freedom safety wat restub skip c...
4,curevac revolutioni mrna f r leb mensch ber be...


In [ ]:
# function for tokenization -> only use when vectorizer doesn't include tokenization (probably included)
#def tokenize(text):
#    tokens = re.split(r'\W+', text)

#    return tokens

# applying function to the column
#df['Content'] = df['Content'].apply(lambda x: tokenize(x))
#df

In [43]:
# Specifiy the path to the folder where you want to save the CSV file
folder_path = '../data/preprocessed'

# Specify the file name and full path for the CSV file
file = 'preprocessed_data.csv'
path = os.path.join(folder_path, file)

# Save the preprocessed data to a CSV file
df.to_csv(path, index=False)

In [47]:
# generate requirements.txt without some weird paths instead of the package version
!pip list --format=freeze > requirements_preprocessing.txt